# Generate the lncRNA based on log2Foldchange

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
from ipywidgets import widgets
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
# Display output text box widget (will populate when value submitted in input)
printmd("**Type the log2fold that you want to use?**")
name=input()    


**Type the log2fold that you want to use?**

-0.58


In [34]:
new_variable = float(name)
printmd("**Cut-off:**")
print(new_variable)

#read files for foldchange criteria
all_files = glob.glob("data/*lncRNA_change.txt")
list_of_dfs = [pd.read_csv(all_files, sep="\t").drop(['Gene', 'baseMean','lfcSE','stat','pvalue', 'padj'], 1).set_index('Transcript') for all_files in all_files]

#loop for creating a list of datadrames - 1 per file
for dataframe, all_file in zip(list_of_dfs, all_files):
    all_file2 = all_file[5:].split('_')
    dataframe.rename(columns={'log2FoldChange':all_file2[0]}, inplace=True)

#Combine all the dfs
combined_df = pd.concat(list_of_dfs, axis=1, sort=True)
#combined_df_pvalue = pd.concat(list_of_dfs2, axis=1, sort=True)

#Create a coloumn named label containing all the cell line intersections based on gene name. Replace "-1" by the wanted foldchange
combined_df['label'] = (combined_df < new_variable).apply(lambda y: combined_df.columns[y.tolist()].tolist(), axis=1)

#Count the number of intersection using "," as delimiter
combined_df['Count Overlap'] = combined_df['label'].astype(str).str.count("\,") +1 

#Sort based on # of intersections
combined_df_sorted = combined_df.sort_values(by=['Count Overlap'], ascending=False)

#remove empty intersection
combined_df_sorted_clean = combined_df_sorted[~combined_df_sorted['label'].astype(str).str.contains('\[]')].fillna(0)
combined_df_sorted_clean

**Cut-off:**

-0.58


,CL40,COLO320,GTG7,HT55,HUTU80,LOVO,LS180,LS411,MDST8,SW1463,label,Count Overlap
ENSG00000279259.1_5,-1.831048,-1.139510,-1.319975,0.000000,0.000000,-0.752096,-2.313718,-1.273999,-1.183875,-1.732485,"[CL40, COLO320, GTG7, LOVO, LS180, LS411, MDST...",8
ENSG00000278730.1_5,-0.829317,-1.284141,-0.157330,-2.984097,1.080155,-0.700020,-0.820394,-1.075129,1.558971,-1.624535,"[CL40, COLO320, HT55, LOVO, LS180, LS411, SW1463]",7
ENSG00000249492.1_6,-1.453492,-0.820529,0.658763,-2.302361,0.000000,-0.817534,-0.758667,-0.705652,0.046443,-2.059978,"[CL40, COLO320, HT55, LOVO, LS180, LS411, SW1463]",7
ENSG00000268858.2_5,0.000000,-0.793413,-0.622039,-0.620857,-0.920734,-0.610275,0.313778,-0.408806,-1.997200,-1.894933,"[COLO320, GTG7, HT55, HUTU80, LOVO, MDST8, SW1...",7
ENSG00000259943.1_5,-1.831048,-0.871624,1.616657,-3.427461,-3.332375,-0.009415,-0.721104,-0.452265,-0.901128,-2.362597,"[CL40, COLO320, HT55, HUTU80, LS180, MDST8, SW...",7
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000245261.1_5,0.000000,1.093942,-0.269868,0.000000,0.000000,0.684741,0.151598,1.127623,-0.610105,1.951066,[MDST8],1
ENSG00000245293.2_6,0.000000,-0.668619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[COLO320],1
ENSG00000245317.2_5,0.000000,-0.268582,0.366016,0.000000,0.000000,0.068297,0.183595,-0.001671,-1.504364,0.000000,[MDST8],1
ENSG00000245556.2_4,0.000000,0.229713,0.003451,0.000000,0.000000,0.496062,-0.660597,1.227454,-0.525235,0.000000,[LS180],1


## Save the matrix as a excel file:

In [18]:
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, sep='\t')
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
csv_download_link(combined_df_sorted_clean, 'matrix_TEAD_lncRNA.csv')   

## Expression of a specific lncRNA

In [1]:
combined_df_sorted_clean.loc["ENSG00000163597.14_2"]

NameError: name 'combined_df_sorted_clean' is not defined